In [52]:
import requests
import pandas as pd
import tushare as ts
import numpy as np
import tqdm
import datetime
import os
import time
import matplotlib.pyplot as plt

# 基础设置
'''
token: tushare通行证
DATE_FORMAT: 日期格式，用于存储文件和参数，默认为 '20200101'
LOWEST_MV: 市值
APPLY_MV: 只更新MV阈值以上的数据和计算指标
'''
token = 'f7a7b0cb0970b6a16628becdc57b070af9583c28ee7b05f858e92b9d'
DATE_FORMAT = "%Y%m%d"
LOWEST_MV = 5e5
APPLY_MV = True


ts.set_token(token)

global pro
pro = ts.pro_api()

## 下载

In [53]:
def update_hist(stock_code):
    
    start = time.time()
    start_date = '20000101'
    
    sub_folder = 'local_hist/' + stock_code.replace('.', '')
    if not os.path.exists(sub_folder):
        os.mkdir(sub_folder)
        
    
    end_date = datetime.datetime.now().strftime(DATE_FORMAT)
    
    
    # 没有就重新下载
    if not os.path.exists(sub_folder + '/' + 'ma.csv'):
        # ma
        # df = ts.pro_bar(ts_code = stock_code, start_date = start_date, end_date = end_date, adj='qfq', ma = ma_lst)
        df = ts.pro_bar(ts_code = stock_code, start_date = start_date, end_date = end_date, adj='qfq')
        df.to_csv(sub_folder + '/' + 'ma.csv')
        print(stock_code + ' ma downloaded.')
        
    # 有的话先找当前日期
    else:
        last_date = str(max(pd.read_csv(sub_folder + '/' + 'ma.csv').trade_date))
        
        if last_date != end_date:
            next_date = (datetime.datetime.strptime(str(last_date), DATE_FORMAT) + \
                         datetime.timedelta(days = 1)).strftime(DATE_FORMAT)
            new_df = ts.pro_bar(ts_code = stock_code, start_date = next_date, end_date = end_date, adj='qfq')

            old_df = pd.read_csv(sub_folder + '/' + 'ma.csv')
            old_df = old_df[[i for i in old_df.columns if 'Unnamed' not in i]]

            new_df = pd.concat([new_df, old_df], axis = 0)
            new_df.to_csv(sub_folder + '/' + 'ma.csv', index = False)

            print(stock_code + ' updated from ' + last_date + ' to ' + end_date)
        else:
            print(stock_code + ' is already updated to ' + end_date)
        
    end = time.time()
    print("Time cost : " + str(end - start) + 's')

## 计算指标

In [54]:
### 均线ma
'''
计算方法：
截取前N个交易日的收盘价计算平均值
'''
def calculate_ma(data, ma_lst = [5, 20, 35, 200]):

    result = data.copy()
    close_price = list(result['close'])
    num_days = len(close_price)
    
    for ma in ma_lst:
        # 初始化为nan
        temp_ma = np.ones(num_days)
        temp_ma.fill(np.nan)
        
        for i in range(num_days - ma):
            temp_ma[i] = np.mean(close_price[i:i+ma])
        
        result['ma_' + str(ma)] = temp_ma
    
    
    return result[['ma_' + str(ma) for ma in ma_lst]]

In [55]:
### 指数均线ema
'''
计算方法：
截取前N个交易日的收盘价计算指数平均值
'''
def calculate_ema(data, N):

    result = data.copy()
    close_price = list(result['close'])
    num_days = len(close_price)
    # N通常为 12 或者 26
    alpha = 2/(N + 1)
    # 至少有一天数据
    assert num_days > 0
    
    # 假设初始值ema1为第一日收盘价
    temp_ema = np.ones(num_days)
    temp_ema[-1] = close_price[-1]
    
    # 从后往前计算，舍去num_days-1的第一天价格
    for i in range(num_days - 2, -1, -1):
        temp_ema[i] = alpha*close_price[i] + (1 - alpha)*temp_ema[i + 1]
        
    result['ema' + str(N)] = temp_ema
    
    return result[['ema' + str(N)]]

In [56]:
'''
计算ema辅助函数
计算
'''
def ema_helper(data, alpha):

    num_days = len(data)
    assert num_days > 0
    
    # 假设初始值ema1为第一日收盘价
    temp_ema = np.ones(num_days)
    temp_ema[-1] = data[-1]
    
    # 从后往前计算，舍去num_days-1的第一天价格
    for i in range(num_days - 2, -1, -1):
        temp_ema[i] = alpha * data[i] + (1 - alpha) * temp_ema[i + 1]
        
    return temp_ema[0]

In [57]:
### 布林线 Boll
'''
计算方法：
中轨线 = 20日的移动平均线（SMA)
上轨线 = 20日的SMA +（20日的标准差 x 2）
下轨线 = 20日的SMA -（20日的标准差 x 2）
'''
def calculate_boll(data, length = 20):
    
    result = data.copy()
    close_price = list(result['close'])
    num_days = len(close_price)
    # 中轨线
    temp_mid = np.ones(num_days)
    temp_mid.fill(np.nan)
    # 上轨线
    temp_upper = np.ones(num_days)
    temp_upper.fill(np.nan)
    # 下轨线
    temp_lower = np.ones(num_days)
    temp_lower.fill(np.nan)
    
    for i in range(num_days - length):
        days_std = np.std(close_price[i:i+length])
        # 20日的移动平均线（SMA)
        temp_mid[i] = np.mean(close_price[i:i+length])
        # 20日的SMA +（20日的标准差 x 2）
        temp_upper[i] = temp_mid[i] + days_std * 2
        # 20日的SMA -（20日的标准差 x 2）
        temp_lower[i] = temp_mid[i] - days_std * 2
    
    
    result['boll_mid'] = temp_mid
    result['boll_upper'] = temp_upper
    result['boll_lower'] = temp_lower
    
    return result[['boll_mid', 'boll_upper', 'boll_lower']]

In [58]:
### KDJ
'''
计算方法：
1. n日RSV＝（Cn-Ln）÷（Hn-Ln）×100 （公式中，Cn为第n日收盘价；Ln为n日内的最低价；Hn为n日内的最高价）
2. 当日K值＝2÷3×前一日K值＋1÷3×当日RSV （若无前一日K值，用50代替）
3. 当日D值＝2÷3×前一日D值＋1÷3×当日K值 （若无前一日D值，用50代替）
4. J值＝3×当日D值-2×当日K值
'''
def calculate_KDJ(data, N = 9):
    
    # 移动平均对于当前数值的权重 current_factor
    current_factor = 1/3
    previous_factor = 1 - current_factor
    
    result = data.copy()
    close_price = list(result['close'])
    high_price = list(result['high'])
    low_price = list(result['low'])
    
    num_days = len(close_price)
    # 未成熟随机值 raw stochastic value
    RSV = np.ones(num_days)
    RSV.fill(50)
    # K线
    K = np.ones(num_days)
    K.fill(50)
    # D线
    D = np.ones(num_days)
    D.fill(50)
    # J线
    J = np.ones(num_days)
    J.fill(50)
    
    for i in range(num_days - 2, -1, -1):
        
        Cn = close_price[i]
        Ln = min(low_price[i:i+N])
        Hn = max(high_price[i:i+N])
        
        current_RSV = 100*((Cn - Ln)/(Hn - Ln))
        previous_RSV = RSV[i + 1]
        previous_K = K[i + 1]
        previous_D = D[i + 1]
        
        current_K = previous_factor * previous_K + current_factor * current_RSV
        current_D = previous_factor * previous_D + current_factor * current_K
        # 百度百科有误，此处修正为J=3K-2D，与各软件统一
        current_J = 3 * current_K - 2 * current_D
        
        RSV[i] = current_RSV
        K[i] = current_K
        D[i] = current_D
        J[i] = current_J
    
    result['RSV'] = RSV
    result['K'] = K
    result['D'] = D
    result['J'] = J
    
    return result[['RSV', 'K', 'D', 'J']]

In [59]:
### RSI
'''
计算方法：
1. 对于某一个时间段N，每天计算上涨U和下跌D：
上涨日 ： U = 价差， D = 0
下跌日 ： U = 0， D = 价差
2. RS = SMMA(U, N) / SMMA(D, N)
3. RSI = 100/(1 + RS)
'''
def calculate_RSI(data, N = 12):
    
    # SMMA指数移动平均对于当前数值的权重 current_factor
    # https://en.wikipedia.org/wiki/Relative_strength_index
    current_factor = 1 / N
    previous_factor = 1 - current_factor
    
    result = data.copy()
    close_price = list(result['close'])
    num_days = len(close_price)
    
    U = np.zeros(num_days)
    D = np.zeros(num_days)
    U_ema = np.zeros(num_days)
    D_ema = np.zeros(num_days)
    RS = np.zeros(num_days)
    RSI = np.zeros(num_days)
    
    for i in range(num_days - 2, -1, -1):
        U[i] = max(close_price[i] - close_price[i + 1], 0)
        D[i] = max(close_price[i + 1] - close_price[i], 0)
        U_ema[i] = previous_factor * U_ema[i + 1] + current_factor * U[i]
        D_ema[i] = previous_factor * D_ema[i + 1] + current_factor * D[i]
        # SMMA(D, n)为0时 RS为100，即一直上涨从未下跌
        if D_ema[i] == 0:
            RS[i] = 100
        else:
            RS[i] = U_ema[i] / D_ema[i]
            
        RSI[i] = 100/(1 + RS[i])
        
    result['RSI_U'] = U
    result['RSI_D'] = D
    result['RSI_U_ema'] = U_ema
    result['RSI_D_ema'] = D_ema
    result['RS'] = RS
    result['RSI'] = RSI
    
    return result[['RSI']]

In [60]:
### MACD
'''
计算方法：
EMA（12）= 前一日EMA（12）×11/13＋今日收盘价×2/13
EMA（26）= 前一日EMA（26）×25/27＋今日收盘价×2/27
DIFF=今日EMA（12）- 今日EMA（26）
DEA= 前一日DEA×8/10＋今日DIF×2/10
BAR (MACD) = 2 × (DIFF－DEA)
=============================
新股上市首日，其DIFF,DEA以及MACD都为0
'''
def calculate_MACD(data, fast_N = 12, slow_N = 26, N = 9):
    
    # https://zhuanlan.zhihu.com/p/41166949
    current_fast_factor = 2 / (1 + fast_N)
    previous_fast_factor = 1 - current_fast_factor
    
    current_slow_factor = 2 / (1 + slow_N)
    previous_slow_factor = 1 - current_slow_factor
    
    current_DEA_factor = 2 /(1 + N)
    previous_DEA_factor = 1 - current_DEA_factor
    
    result = data.copy()
    close_price = list(result['close'])
    num_days = len(close_price)
    
    ema_fast = np.zeros(num_days)
    ema_slow = np.zeros(num_days)
    ema_diff = np.zeros(num_days)
    DEA = np.zeros(num_days)
    MACD = np.zeros(num_days)
    #初始化
    ema_fast[-1] = close_price[-1]
    ema_slow[-1] = close_price[-1]
    
    for i in range(num_days - 2, -1, -1):
        ema_fast[i] = current_fast_factor * close_price[i] + previous_fast_factor * ema_fast[i + 1] 
        ema_slow[i] = current_slow_factor * close_price[i] + previous_slow_factor * ema_slow[i + 1]
        ema_diff[i] = ema_fast[i] - ema_slow[i]
        DEA[i] = current_DEA_factor * ema_diff[i] + previous_DEA_factor * DEA[i + 1] 
        MACD[i] = 2 * (ema_diff[i] - DEA[i])
        
    result['MACD_fast_ema'] = ema_fast
    result['MACD_slow_ema'] = ema_slow
    result['MACD_diff_ema'] = ema_diff
    result['MACD_DEA'] = DEA
    result['MACD'] = MACD
    
    return result[['MACD_fast_ema', 'MACD_slow_ema', 'MACD', 'MACD_diff_ema', 'MACD_DEA']]

In [61]:
def build_stats(stock_code_list, ma_lst = [5, 10, 20, 30, 35, 45, 60, 90, 120, 130, 200, 250]):
    
    start = time.time()
    for stock_code in tqdm.tqdm(stock_code_list):
        
        sub_folder = 'local_hist/' + stock_code.replace('.', '')
        
        if not os.path.exists(sub_folder + '/' + 'ma.csv'):
            continue
        
        # 如果有数据未下载，则跳过
        ori_data = pd.read_csv(sub_folder + '/' + 'ma.csv')
        ori_data = ori_data[~pd.isnull(ori_data.close)]
        stats_ma = calculate_ma(ori_data, ma_lst = ma_lst)
        stats_boll = calculate_boll(ori_data, length = 20)
        stats_KDJ = calculate_KDJ(ori_data, N = 20)
        stats_RSI = calculate_RSI(ori_data, N = 12)
        stats_MACD = calculate_MACD(ori_data, fast_N = 12, slow_N = 26)
        
        stats_combined = pd.concat([ori_data, stats_ma, stats_boll, stats_KDJ, stats_RSI, stats_MACD], axis = 1)
        stats_combined.to_csv(sub_folder + '/' + 'stats.csv')
    
    end = time.time()
    print("="*60)
    print("Eplased Time for Calculating Stats: " + str(end - start) + 's')
    print("="*60)

# 下载所有股票市值数据以及历史数据

In [62]:
def download_update(stock_code_list):

    start = time.time()
    # 更新所有股票数据
    for stock_code in tqdm.tqdm(stock_code_list):
        update_hist(stock_code)
    
    end = time.time()
    print("="*60)
    print("Total Time Cost for Downloading : " + str(end - start) + 's')
    print("="*60)

In [63]:
def main():
    # 先更新数据
    try:
        all_stock_total_mv = pro.query('daily_basic', ts_code='', trade_date = current_date, 
                                fields='ts_code,trade_date,turnover_rate,volume_ratio,pe,pb,total_mv')
        all_stock = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
        all_stock_total_mv.to_csv('all_stock_total_mv.csv')
        print("update market value from tushare")
    except:
        all_stock_total_mv = pd.read_csv('all_stock_total_mv.csv')
        all_stock = pd.read_csv("all_stock.csv")
        print("load market value from local")
    
    if not os.path.exists('local_hist'):
        os.makedirs('local_hist')
    

    if APPLY_MV:
        # 需要更新的股票数据，设置市值阈值
        concerned_stocks = list(all_stock_total_mv[all_stock_total_mv.total_mv >= LOWEST_MV].ts_code)
        concerned_stocks_df = all_stock[all_stock.ts_code.isin(concerned_stocks)]
        concerned_stocks_pairs = [tuple((code, name, industry)) for code, name, industry in \
                                  zip(concerned_stocks_df.ts_code, concerned_stocks_df.name, concerned_stocks_df.industry)]
        # 执行
        download_update(concerned_stocks)
        build_stats(concerned_stocks)
    else:
        download_update(all_stock.ts_code)
        build_stats(all_stock.ts_code)

In [64]:
main()

load market value from local



  0%|          | 0/2129 [00:00<?, ?it/s]

002135.SZ updated from 20210114 to 20210115
Time cost : 26.47813892364502s



  0%|          | 1/2129 [00:26<15:39:07, 26.48s/it]

603806.SH updated from 20210114 to 20210115
Time cost : 7.15948224067688s


...

 77%|███████▋  | 1637/2129 [15:50<04:31,  1.81it/s]


 85%|████████▌ | 1819/2129 [17:37<02:56,  1.76it/s]


 94%|█████████▍| 1999/2129 [19:15<00:58,  2.24it/s]


100%|██████████| 2129/2129 [20:31<00:00,  1.73it/s]


Eplased Time for Calculating Stats: 1231.147925376892s
